# Random Forest with GridSearch

In [37]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score
import pickle
import pathlib


In [38]:

DATA_DIR = pathlib.Path.cwd().parent / 'data'
clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

# Lodando
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Data columns (total 70 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   MS.SubClass      2877 non-null   category
 1   MS.Zoning        2877 non-null   category
 2   Lot.Frontage     2877 non-null   float64 
 3   Lot.Area         2877 non-null   float64 
 4   Lot.Shape        2877 non-null   category
 5   Land.Contour     2877 non-null   category
 6   Lot.Config       2877 non-null   category
 7   Land.Slope       2877 non-null   category
 8   Neighborhood     2877 non-null   category
 9   Bldg.Type        2877 non-null   category
 10  House.Style      2877 non-null   category
 11  Overall.Qual     2877 non-null   category
 12  Overall.Cond     2877 non-null   category
 13  Roof.Style       2877 non-null   category
 14  Mas.Vnr.Type     2877 non-null   category
 15  Mas.Vnr.Area     2877 non-null   float64 
 16  Exter.Qual       2877 non-null   category
 17  

In [39]:
# Target vs Features
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

# Test + Train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((2301, 69), (576, 69), (2301,), (576,))

In [40]:
# Identificar colunas categóricas
categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == 'category' and 
                    X_train[cname].nunique() < 10]

categorical_cols


['MS.Zoning',
 'Lot.Shape',
 'Land.Contour',
 'Lot.Config',
 'Land.Slope',
 'Bldg.Type',
 'House.Style',
 'Overall.Cond',
 'Roof.Style',
 'Mas.Vnr.Type',
 'Exter.Qual',
 'Exter.Cond',
 'Foundation',
 'Bsmt.Qual',
 'Bsmt.Cond',
 'Bsmt.Exposure',
 'BsmtFin.Type.1',
 'BsmtFin.Type.2',
 'Heating.QC',
 'Central.Air',
 'Electrical',
 'Kitchen.Qual',
 'Functional',
 'Garage.Type',
 'Garage.Finish',
 'Paved.Drive',
 'Fence',
 'Sale.Type',
 'Sale.Condition',
 'Condition']

In [41]:
# One Hot Encoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols)  # drop='first' to avoid collinearity
    ],
    remainder='passthrough'  # automatically passthrough remaining columns
)

# Pipando o line
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('rf', RandomForestRegressor(random_state=42))
                       ])

# Hyperparametros
param_grid = {
    'rf__n_estimators': [10, 50, 100, 200],
    'rf__max_depth': [None, 10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

# Gridsearch
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', 
                           verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=10; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.0s
[CV] END rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time= 

ValueError: 
All the 720 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
144 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/lib/python3.11/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/femcdias/Documents/Insper/Semestre 4/Machine Learning/Projeto1/ames/env/lib/python3.11/site-packages/sklearn/pipeline.py", line 427, in fit

In [ ]:

# Grid search
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

y_pred = grid_search.best_estimator_.predict(X_test)

# RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Mean Squared Error on the testing set: {mse:.2f}")
print(f"Root Mean Squared Error on the testing set: {rmse:.2f}")
